# Teste de Hipótese

Considere o banco de dados `aluguel_bicicleta.csv` contendo informações sobre vários aluguéis de bicicleta em um determinado período. Queremos  verificar o seguinte: **o aluguel de bicicletas é dependente dos dias úteis ou não?** Em outras palavras, queremos saber se o número de bicicleta alugadas em dias de trabalho é diferente do número de bicicletas alugadas aos finais de semana.

### Definindo as hipóteses:

* $H_0$: $\mu_1 = \mu_2$  (Hipótese nula) 
* $H_1$: $\mu_1 > \mu_2$  (Hipótese alternativa)

Definindo que $\mu_1$ é a média de aluguéis de bicicleta em dias úteis e que $\mu_2$ é a média de aluguéis de bicicleta em fins de semana.


---

Abaixo importamos as bibliotecas que utilizaremos para lidar com os dados e executar os cálculos necessários.

In [1]:
import pandas as pd                 #biblioteca pra lidar com os dados
import datetime as dt               #biblioteca pra mexer com as datas/horários da tabela
import numpy as np                  #biblioteca para contas
import matplotlib.pyplot as plt     #biblioteca de plotar gráficos
from scipy import stats             #biblioteca de estatistica

No próximo trecho recebemos os dados e filtramos apenas aqueles que são relevantes para este problema.

In [2]:
lista_base = pd.read_csv('aluguel_bicicleta.csv', delimiter=',')    #recebe dados
lista_base = lista_base.loc[:,['datetime','workingday','count']]    #filtra colunas

As informações de tempo dos dados estão discriminadas por dia e horário. Para este trabalho apenas os dias são relevantes. O próximo trecho de código agrupa as informações por dia.

In [3]:
lista_base['datetime'] = pd.to_datetime(lista_base['datetime'])                     #converte para o tipo de dado "datetime"
dates = lista_base['datetime'].dt.floor('D')                                        #agrupa os dias
lista_dias = lista_base.groupby(dates)['workingday','count'].sum().reset_index()    #soma os alugueis de cada hora por dia

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  This is separate from the ipykernel package so we can avoid doing imports until


Dividindo os dados em duas subtabelas: dias úteis e dias não-úteis.

In [4]:
workingdays = lista_dias[lista_dias['workingday'] != 0].loc[:,['count']]     #dados dos dias úteis
nonworkdays = lista_dias[lista_dias['workingday'] == 0].loc[:,['count']]     #dados dos dias inúteis

Fazendo o teste de Levene sobre as duas amostras.

In [5]:
stat, p = stats.levene(workingdays['count'],nonworkdays['count'])   #teste de levene
print(stat,p)

1.1696924154089139 0.28003858261286085


Agora vamos calcular o desvio padrão de cada amostra:

$$S = \sqrt{\frac{\sum\limits_{i=1}^{n}(x_i - M)^2}{n}}$$

* $S$ é o desvio padrão

* $x_i$ é cada elemento da amostra com $n$ elementos

* $M$ é a média aritmética das amostras


No código abaixo `S1` é o desvio padrão dos dias úteis e `S2` é o desvio padrão dos dias não-úteis.

In [7]:
#Desvio Padrão da Amostra dos dias úteis
M1 = workingdays['count'].mean()        #média de alugueis dos dias úteis
n1 = len(workingdays['count'])          #quantidade de dias úteis
S1 = 0
for x in workingdays['count']:          #somatorio
    S1 = S1 + np.power((x - M1),2)
S1 = S1/n1                              #desvio padrão S1 ao quadrado
S1 = np.sqrt(S1)
print(S1)


#Desvio Padrão da Amostra dos dias inúteis
M2 = nonworkdays['count'].mean()        #média de alugueis dos dias inúteis
n2 = len(nonworkdays['count'])          #quantidade de dias inúteis
S2 = 0
for x in nonworkdays['count']:          #somatorio
    S2 = S2 + np.power((x - M2),2)
S2 = S2/n2                              #desvio padrão S2 ao quadrado
S2 = np.sqrt(S2)
print(S2)

1826.1223267476225
1949.6338849087294


Calcula a estimativa

$$ S = \frac{(n_1 - 1)S_1 + (n_2 - 1)S_2}{n_1 + n_2 -2}$$

* $S$ é a variância agrupada (pooled variance)
* $S_1$ é o desvio padrão da amostra 1 e $S_2$ é o desvio padrão da amostra 2
* $n_1$ e $n_2$ é a quantidade de elementos da amostra 1 e 2, respectivamente

In [12]:
df = (n1 + n2 -2)                           #grau de liberdade
S  = (((n1 - 1)*S1) + ((n2 - 1)* S2))/df    #variância agrupada (quando a amostra tem variância igual)
print(S)

1865.2977989396916


Fazendo o t-teste:

$$
    t = \frac{M_1 - M_2}{\sqrt{\frac{S_1^2}{n_1} + \frac{S_2^2}{n_2}}}
$$

In [11]:
t = (M1 - M2)/np.sqrt((pow(S1,2)/n1) + (pow(S2,2)/n2))  #calcula o t-teste

print(t)

0.4352691641055156


In [ ]:
#pegando o valor base da tabela para o grau de liberdade deste problema   
stats.t.ppf(q=.05,df=df)        #right tailed test

-1.648216847484026